In [1]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from string import punctuation

In [2]:
tweet_df = pd.read_csv("train.csv", encoding="latin")

In [3]:
tweet_df.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


Sentiment :
0 = negative
1 = positive

In [3]:
ps = PorterStemmer()
negation_list = ["arent","isnt","dont","doesnt","not","cant","couldnt", "werent",
                 "wont","didnt","never","nothing","nowhere","noone","none"
                "hasnt","hadnt","shouldnt","wouldnt","aint"]
               
def preProcessTweets(tweet):
    tweet = tweet.lower()
    tweet = re.sub('n[^A-Za-z ]t','nt', tweet)
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', tweet)
    tweet = re.sub('@[^\s]+', '', tweet)
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = word_tokenize(tweet)
    tweet_list = [];
    negate = False
    for word in tweet:
        word = ps.stem(word)
        if word in negation_list:
            negate = True
        elif negate is True and word in list(punctuation):
            negate = False
            
        if negate and word not in negation_list:
            word = "not_"+word
        else:
            pass
        word = re.sub('[^A-Za-z_ ]+', '', word)
        if len(word) > 2 and word not in stopwords.words('english'):
            tweet_list.append(word)
    tweet_set = set(tweet_list)
    return " ".join(tweet_set)

1. tweet.lower() is to change the tweet to lowercase
1. re.sub('n[^A-Za-z ]t','nt', tweet) is regex for removing special character between 'n' and 't' letter. This syntax is used to normalize the negation text so can be matched with negation_list
2. re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', tweet) is to remove URL from tweet
3. re.sub('@[^\s]+', '', tweet) is to remove username/mention from tweet
4. re.sub(r'#([^\s]+)', r'\1', tweet) is to remove hashtag before text
5. re.sub('[^A-Za-z_ ]+', '', word) is to remove any special character
6. word_tokenize is to tokenize the tweet into a list
7. ps.stem(word) is to stem the word into root/base form

In [5]:
tweet_df["clean_text"] = tweet_df["SentimentText"].apply(preProcessTweets)

In [6]:
tweet_df.head()

,ItemID,Sentiment,SentimentText,clean_text
0,1,0,is so sad for my APL frie...,apl sad friend
1,2,0,I missed the New Moon trail...,trailer moon miss new
2,3,1,omg its already 7:30 :O,omg alreadi
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...,sooo sinc crown thi dentist gunna get put cri ...
4,5,0,i think mi bf is cheating on me!!! ...,think t_t cheat


In [7]:
tweet_df.to_csv("train_preprocessed.csv")